In [1]:
import pandas as pd
import requests
import os
from bs4 import BeautifulSoup
pd.set_option('display.max_rows', None)

In [2]:
headers = {'user-agent': 'Mozilla/5.0'}
resposta = requests.get('https://www.camarabotucatu.sp.gov.br/Vereador',
                        headers= headers
                        )

## Pegando o Nome e o Partido dos Vereadores

In [3]:
text_estruturado = BeautifulSoup(resposta.text)
#text_estruturado

In [4]:
lista_vereadores = text_estruturado.find_all('h2')

In [5]:
lista_vereadores

[<h2>
 <a href="http://www.camarabotucatu.sp.gov.br/Consulta/vereadores/Vereador/68" target="_blank">Abelardo Wanderlino da Costa Neto - Abelardo</a>
 </h2>,
 <h2>
 <a href="http://www.camarabotucatu.sp.gov.br/Consulta/vereadores/Vereador/81" target="_blank">Alessandra Lucchesi de Oliveira - Alessandra Lucchesi</a>
 </h2>,
 <h2>
 <a href="http://www.camarabotucatu.sp.gov.br/Consulta/vereadores/Vereador/69" target="_blank">André Rogério Barbosa - Curumim (Licenciado)</a>
 </h2>,
 <h2>
 <a href="http://www.camarabotucatu.sp.gov.br/Consulta/vereadores/Vereador/47" target="_blank">Antonio Carlos Vaz de Almeida - Cula</a>
 </h2>,
 <h2>
 <a href="http://www.camarabotucatu.sp.gov.br/Consulta/vereadores/Vereador/7076" target="_blank">Cláudia Maria Gabriel - Cláudia Gabriel (Licenciada)</a>
 </h2>,
 <h2>
 <a href="http://www.camarabotucatu.sp.gov.br/Consulta/vereadores/Vereador/7075" target="_blank">Elias Marcelo Sleiman - Marcelo Sleiman</a>
 </h2>,
 <h2>
 <a href="http://www.camarabotucatu.sp

In [6]:
# Captando o nome e o partido dos vereadores
tags_li = text_estruturado.find_all('span')
lista_partidos = []
for i in range(len(tags_li)):
    if tags_li[i].find_all(['li']):
        if tags_li[i].find_all('a'):
            lista_partidos.append( tags_li[i].text.replace('\n', '') )


nomes_vereadores = []
for i in range(len(lista_vereadores)):
    #print(i)
    nomes_vereadores.append(  lista_vereadores[i].contents[1].text.split('-')[1].strip()  )


'testando o spli'

## Captando as imagens dos Vereadores

In [7]:
import re

teste = []
teste2 = []

x = text_estruturado.find_all('div', {'class': 'col-md-3'})
for i in range(len(x)):
    if x[i].find('div', {'class':'img'}):
        teste.append(x[i].find('div', {'class':'img'}))
        #print(teste)
for j in range(len(teste)):
    if teste[j].find('img'):
        teste2.append(teste[j].find_all('img', {'src': re.compile('\.jpg$', re.IGNORECASE)}))


In [8]:
save_folder = "../img"
for img_tag in teste2:
    src = img_tag[0]['src']
    alt = img_tag[0]['alt'].split('-')[1].strip()
    img_url = f"https://www.camarabotucatu.sp.gov.br/{src}" 
    filename = os.path.join(save_folder, f"{alt}.jpg")

    response = requests.get(img_url)
    with open(filename, 'wb') as f:
        f.write(response.content)

In [9]:
df_vereadores = pd.DataFrame()

In [10]:
lista_partidos

['Republicanos',
 'PSDB-Partido da Social Democracia Brasileira',
 'PSDB-Partido da Social Democracia Brasileira',
 'PSDB-Partido da Social Democracia Brasileira',
 'União Brasil',
 'União Brasil',
 'Republicanos',
 'União Brasil',
 'PSDB-Partido da Social Democracia Brasileira',
 'PSDB-Partido da Social Democracia Brasileira',
 'União Brasil',
 'PDT-Partido Democrático Trabalhista',
 'Republicanos']

In [11]:
df_vereadores['nome_vereador'] = nomes_vereadores
df_vereadores['partido_vereador'] = lista_partidos

In [12]:
df_vereadores.dtypes

nome_vereador       object
partido_vereador    object
dtype: object

In [13]:
df_vereadores

,nome_vereador,partido_vereador
0,Abelardo,Republicanos
1,Alessandra Lucchesi,PSDB-Partido da Social Democracia Brasileira
2,Curumim (Licenciado),PSDB-Partido da Social Democracia Brasileira
3,Cula,PSDB-Partido da Social Democracia Brasileira
4,Cláudia Gabriel (Licenciada),União Brasil
5,Marcelo Sleiman,União Brasil
6,Erika da Liga do Bem,Republicanos
7,Pedroso (Suplente),União Brasil
8,Sargento Laudo,PSDB-Partido da Social Democracia Brasileira
9,Lelo Pagani (Suplente),PSDB-Partido da Social Democracia Brasileira


## Para Cada Vereador trazer tabela do que ja fez em cada Ano

In [14]:
links_vereadores = []
for i in range(len(lista_vereadores)):
    links_vereadores.append(lista_vereadores[i].a.get('href'))

df_vereadores['link'] = links_vereadores

In [15]:
df_final = pd.DataFrame(columns=['Tipo', 'Quantidade','Ano'])

for i in range(len(df_vereadores)):

    print(i)

    #pega o nome do vereador
    nome_atual_vereador = df_vereadores.iloc[i][:i+1]['nome_vereador']


    #pega o link do vereador
    #df_vereadores.iloc[i][:i+1]['link']


    headers = {'user-agent': 'Mozilla/5.0'}
    resposta_vereador = requests.get(df_vereadores[:i+1]['link'].iloc[-1],
                        headers= headers
                        )
    
    vereador_estruturado = BeautifulSoup(resposta_vereador.text)
    tabela_documentos = vereador_estruturado.find_all('table')

    # Pegando a linha da tabela do site que contém os anos de atuação dos vereadores
    lista_anos = []

    for i in range(len(tabela_documentos)):
        if tabela_documentos[i].find_all('caption')[0].text == 'Documentos':
            tabela_documentos = tabela_documentos[i]


    for i in range(len( tabela_documentos.find_all('th', {'class': 'text-right'}) ) ):
        if tabela_documentos.find_all('th', {'class': 'text-right'})[i].text:
            lista_anos.append(tabela_documentos.find_all('th', {'class': 'text-right'})[i].text)
    lista_anos = sorted([*set(lista_anos)])
    if 'Total' in lista_anos:
        lista_anos.remove('Total')
    #lista_anos


    # Pegando todas as proposituras do vereador em cada tipo e alocando em um dataframe
    tbody = tabela_documentos.find_all('tbody')[0].find_all('tr')#[0]#.find_all('td')
    list_dict = []
    df_teste = pd.DataFrame(columns=['Tipo', 'Quantidade'])

    for i in range(len(tbody)):
        for j in range(len( tbody[i].find_all('td') )):
            list_dict.append(tbody[i].find_all('td')[j].text)
        #print(list_dict)
        
        #list_aux = list_dict
        #list_aux.remove(list_aux[0])
        df_novo = pd.DataFrame({'Tipo': [list_dict[0]]*(len(list_dict)-2), 'Quantidade': list_dict[1:-1], 'Ano': lista_anos,'Vereador': nome_atual_vereador})
        df_teste = pd.concat([df_teste, df_novo], ignore_index=True)

        list_dict = []
    df_final = pd.concat([df_final,df_teste])
    df_final = df_final.loc[df_final['Tipo'] != 'Total']
    
    

0
1
2
3
4
5
6
7
8
9
10
11
12


In [16]:
df_final['Quantidade'] = df_final['Quantidade'].apply(lambda x: 0 if x == '-' else x)
df_final.head()

,Tipo,Quantidade,Ano,Vereador
0,Correspondências Recebidas,0,2009,Abelardo
1,Correspondências Recebidas,1,2010,Abelardo
2,Correspondências Recebidas,0,2011,Abelardo
3,Correspondências Recebidas,0,2012,Abelardo
4,Correspondências Recebidas,0,2017,Abelardo


In [17]:
df_final['Quantidade'] = df_final['Quantidade'].astype('int')
df_final['Ano'] = df_final['Ano'].astype('str')
df_final.dtypes

Tipo          object
Quantidade     int32
Ano           object
Vereador      object
dtype: object

## Salvando no Banco de Dados

In [18]:
import psycopg2

dbname = "prefeitura"
user = "postgres"
password = "96047991"
host = "localhost"
port = "5432"

conn = psycopg2.connect(
    host=host,
    port=port,
    user=user,
    password=password
)

#conn.set_isolation_level(0)
#cur = conn.cursor()
#cur.execute('CREATE DATABASE prefeitura')
#conn.commit()
#cur.close()
#conn.close()



In [19]:
conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port)

In [33]:
cur = conn.cursor()
cur.execute('DROP TABLE imagens_vereadores')

In [21]:
df_final.dtypes

Tipo          object
Quantidade     int32
Ano           object
Vereador      object
dtype: object

In [22]:
cur = conn.cursor()

#cur.execute("CREATE TABLE vereadores (nome_vereador VARCHAR(50), partido_vereador VARCHAR(50), link_vereador VARCHAR(200) );")
#cur.execute("CREATE TABLE proposituras (Tipo VARCHAR(50), Quantidade INTEGER, Ano VARCHAR(50), Vereador VARCHAR(50));")

#cur.execute("INSERT INTO tabela1 (coluna1, coluna2) VALUES (%s, %s);", (10, "texto"))
#cur.execute("INSERT INTO tabela2 (coluna1, coluna2) VALUES (%s, %s);", ("texto", 3.14))

#conn.commit()

#cur.close()
#conn.close()

In [23]:
from sqlalchemy import create_engine

engine = create_engine('postgresql://postgres:'+password+'@'+host+':'+port+'/'+dbname)
#df_vereadores.to_sql('vereadores', engine, if_exists='replace', index=False)
df_final.to_sql('proposituras', engine, if_exists='replace', index=False)

470

('Abelardo.jpg', <memory at 0x00000259BF843400>)
('Alessandra Lucchesi.jpg', <memory at 0x00000259BF843280>)
('Cláudia Gabriel (Licenciada).jpg', <memory at 0x00000259BF843100>)
('Cula.jpg', <memory at 0x00000259BE3CDB80>)
('Curumim (Licenciado).jpg', <memory at 0x00000259BE3CD940>)
('Erika da Liga do Bem.jpg', <memory at 0x00000259BE3CDA00>)
('Lelo Pagani (Suplente).jpg', <memory at 0x00000259BE3CD040>)
('Marcelo Sleiman.jpg', <memory at 0x00000259BE3CDE80>)
('PALHINHA.jpg', <memory at 0x00000259BE3CDF40>)
('Pedroso (Suplente).jpg', <memory at 0x00000259BE3CDC40>)
('Rose Ielo.jpg', <memory at 0x00000259BE3CDDC0>)
('Sargento Laudo.jpg', <memory at 0x00000259BE3CD880>)
('Silvio.jpg', <memory at 0x00000259C0331280>)


In [34]:
## TABELA DE IMAGENS DOS VEREADORES!!!
cur = conn.cursor()

cur.execute('''
    CREATE TABLE imagens_vereadores (
        vereador VARCHAR(50),
        imagem BYTEA
    );
''')

In [29]:
import os
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

In [35]:
diretorio = "C:/Users/thale/Documents/Estudos/Projeto Prefeitura"
for arquivo in os.listdir(diretorio):
    if arquivo.endswith(".jpg") or arquivo.endswith(".png"):
        with open(os.path.join(diretorio, arquivo), "rb") as imagem_arquivo:
            conteudo_imagem = imagem_arquivo.read()
            arquivo1 = arquivo.split('.')[0]
            cur.execute("INSERT INTO imagens_vereadores (vereador, imagem) VALUES (%s, %s)", (arquivo1, psycopg2.Binary(conteudo_imagem)))
            conn.commit()
        print(arquivo)

Abelardo.jpg
Alessandra Lucchesi.jpg
Cláudia Gabriel (Licenciada).jpg
Cula.jpg
Curumim (Licenciado).jpg
Erika da Liga do Bem.jpg
Lelo Pagani (Suplente).jpg
Marcelo Sleiman.jpg
PALHINHA.jpg
Pedroso (Suplente).jpg
Rose Ielo.jpg
Sargento Laudo.jpg
Silvio.jpg


In [36]:
cur = conn.cursor()
cur.execute('SELECT * FROM imagens_vereadores')

results = cur.fetchall()
for row in results:
    print(row)

('Abelardo', <memory at 0x000002547862C580>)
('Alessandra Lucchesi', <memory at 0x000002547862C640>)
('Cláudia Gabriel (Licenciada)', <memory at 0x000002547862C700>)
('Cula', <memory at 0x000002547862C7C0>)
('Curumim (Licenciado)', <memory at 0x000002547862C880>)
('Erika da Liga do Bem', <memory at 0x000002547862C940>)
('Lelo Pagani (Suplente)', <memory at 0x000002547862CA00>)
('Marcelo Sleiman', <memory at 0x000002547862CAC0>)
('PALHINHA', <memory at 0x000002547862CB80>)
('Pedroso (Suplente)', <memory at 0x000002547862CC40>)
('Rose Ielo', <memory at 0x000002547862CD00>)
('Sargento Laudo', <memory at 0x000002547862CDC0>)
('Silvio', <memory at 0x000002547862CE80>)
